In [1]:
!pip install git+https://github.com/erkil1452/gaze360/tree/master/code


  Cloning https://github.com/erkil1452/gaze360/tree/master/code to c:\users\108652706\appdata\local\temp\pip-req-build-db8tkuga


  Running command git clone -q https://github.com/erkil1452/gaze360/tree/master/code 'C:\Users\108652706\AppData\Local\Temp\pip-req-build-db8tkuga'
  fatal: repository 'https://github.com/erkil1452/gaze360/tree/master/code/' not found
ERROR: Command errored out with exit status 128: git clone -q https://github.com/erkil1452/gaze360/tree/master/code 'C:\Users\108652706\AppData\Local\Temp\pip-req-build-db8tkuga' Check the logs for full command output.


In [2]:
import torch.utils.data as data
from PIL import Image
import os
import os.path
import torchvision.transforms as transforms
import torch
import numpy as np
import re
import glob
import random
import cv2
import torch.nn as nn
import math
import random
import scipy.io as sio

IMG_EXTENSIONS = [
    '.jpg', '.JPG', '.jpeg', '.JPEG',
    '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP',
]

def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

def make_dataset(source_path,file_name):
    images = []
    print(file_name)
    with open(file_name, 'r') as f:
        for line in f:
            line = line[:-1]
            line = line.replace("\t", " ")
            line = line.replace("  ", " ")
            split_lines = line.split(" ")
            if(len(split_lines)>3):
                frame_number = int(split_lines[0].split('/')[-1][:-4])
                lists_sources = []
                for j in range(-3,4):
                    name_frame = '/'.join(split_lines[0].split('/')[:-1]+['%0.6d.jpg'%(frame_number+j)])
                    name = '{0}/{1}'.format(source_path, name_frame)
                    lists_sources.append(name)

                gaze = np.zeros((3))
           
                gaze[0] = float(split_lines[1])
                gaze[1] = float(split_lines[2])
                gaze[2] = float(split_lines[3])
                item = (lists_sources,gaze)
                images.append(item)
    return images


def default_loader(path):
    try:
        im = Image.open(path).convert('RGB')
        return im
    except OSError:
        print(path)
        return Image.new("RGB", (512, 512), "white")




class ImagerLoader(data.Dataset):
    def __init__(self, source_path,file_name,
                transform=None, target_transform=None, loader=default_loader):

        imgs = make_dataset(source_path,file_name)

        self.source_path = source_path
        self.file_name = file_name

        self.imgs = imgs
        self.transform = transform
        self.target_transform = transform
        self.loader = loader


    def __getitem__(self, index):
        path_source,gaze = self.imgs[index]


        gaze_float = torch.Tensor(gaze)
        gaze_float = torch.FloatTensor(gaze_float)
        normalized_gaze = nn.functional.normalize(gaze_float.view(1,3)).view(3)

        source_video = torch.FloatTensor(7,3,224,224)
        for i,frame_path in enumerate(path_source):
            source_video[i,...] = self.transform(self.loader(frame_path))

        source_video = source_video.view(21,224,224)

        spherical_vector = torch.FloatTensor(2)
        spherical_vector[0] = math.atan2(normalized_gaze[0],-normalized_gaze[2])
        spherical_vector[1] = math.asin(normalized_gaze[1])
        return source_video,spherical_vector


        
    def __len__(self):
        return len(self.imgs)

In [3]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import numpy as np
from torch.nn.init import normal, constant
import math


#resnet18 = models.resnet18(pretrained=True)


class GazeLSTM(nn.Module):
    def __init__(self):
        super(GazeLSTM, self).__init__()
        self.img_feature_dim = 256  # the dimension of the CNN feature to represent each frame

        self.base_model = models.resnet18(pretrained=True)

        self.base_model.fc2 = nn.Linear(1000, self.img_feature_dim)

        self.lstm = nn.LSTM(self.img_feature_dim, self.img_feature_dim,bidirectional=True,num_layers=2,batch_first=True)

        # The linear layer that maps the LSTM with the 3 outputs
        self.last_layer = nn.Linear(2*self.img_feature_dim, 3)


    def forward(self, input):

        base_out = self.base_model(input.view((-1, 3) + input.size()[-2:]))

        base_out = base_out.view(input.size(0),7,self.img_feature_dim)

        lstm_out, _ = self.lstm(base_out)
        lstm_out = lstm_out[:,3,:]
        output = self.last_layer(lstm_out).view(-1,3)


        angular_output = output[:,:2]
        angular_output[:,0:1] = math.pi*nn.Tanh()(angular_output[:,0:1])
        angular_output[:,1:2] = (math.pi/2)*nn.Tanh()(angular_output[:,1:2])

        var = math.pi*nn.Sigmoid()(output[:,2:3])
        var = var.view(-1,1).expand(var.size(0),2)

        return angular_output,var
class PinBallLoss(nn.Module):
    def __init__(self):
        super(PinBallLoss, self).__init__()
        self.q1 = 0.1
        self.q9 = 1-self.q1

    def forward(self, output_o,target_o,var_o):
        q_10 = target_o-(output_o-var_o)
        q_90 = target_o-(output_o+var_o)

        loss_10 = torch.max(self.q1*q_10, (self.q1-1)*q_10)
        loss_90 = torch.max(self.q9*q_90, (self.q9-1)*q_90)


        loss_10 = torch.mean(loss_10)
        loss_90 = torch.mean(loss_90)

        return loss_10+loss_90

In [4]:
import argparse
import os
import shutil
import time
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
#from data_loader import ImagerLoader
import numpy as np
import random
import math
import torchvision.utils as vutils
#from model import GazeLSTM,PinBallLoss

source_path = "/imgs/"
val_file = "validation.txt"
train_file = "train.txt"

test_file = "test.txt"


workers = 7;
epochs = 20
batch_size = 10
best_error = 100 # init with a large value
lr = 1e-4

test = False
checkpoint_test = 'gaze360_model.pth.tar'
network_name = 'Gaze360'

from tensorboardX import SummaryWriter
foo = SummaryWriter(comment=network_name)


count_test = 0
count = 0




def main():
    global args, best_error

    #model_v = GazeLSTM()
    #model = torch.nn.DataParallel(model_v).cuda()
    #model.cuda()


    #cudnn.benchmark = True

    image_normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    """

    train_loader = torch.utils.data.DataLoader(
        ImagerLoader(source_path,train_file,transforms.Compose([
            transforms.RandomResizedCrop(size=224,scale=(0.8,1)),transforms.ToTensor(),image_normalize,
        ])),
        batch_size=batch_size, shuffle=True,
        num_workers=workers, pin_memory=True)
    """
    val_loader = torch.utils.data.DataLoader(
        ImagerLoader(source_path,val_file,transforms.Compose([
            transforms.Resize((224,224)),transforms.ToTensor(),image_normalize,
        ])),
        batch_size=batch_size, shuffle=True,
        num_workers=workers, pin_memory=True)



    #criterion = PinBallLoss().cuda()

    #optimizer = torch.optim.Adam(model.parameters(), lr)

    if test==True:

        test_loader = torch.utils.data.DataLoader(
            ImagerLoader(source_path,test_file,transforms.Compose([
                transforms.Resize((224,224)),transforms.ToTensor(),image_normalize,
            ])),
            batch_size=batch_size, shuffle=True,
            num_workers=workers, pin_memory=True)
        checkpoint = torch.load(checkpoint_test,map_location=torch.device('cpu'))
        model.load_state_dict(checkpoint['state_dict'])
        angular_error = validate(test_loader, model, criterion)
        print('Angular Error is',angular_error)

"""
    for epoch in range(0, epochs):


        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch)

        # evaluate on validation set
        angular_error = validate(val_loader, model, criterion)

        # remember best angular error in validation and save checkpoint
        is_best = angular_error < best_error
        best_error = min(angular_error, best_error)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec1': best_error,
        }, is_best)
"""
def validate(val_loader, model, criterion):
    global count_test
    batch_time = AverageMeter()
    losses = AverageMeter()
    prediction_error = AverageMeter()
    model.eval()
    end = time.time()
    angular = AverageMeter()

    for i, (source_frame,target) in enumerate(val_loader):

        source_frame = source_frame.cuda(non_blocking=True)
        target = target.cuda(non_blocking=True)

        source_frame_var = torch.autograd.Variable(source_frame,volatile=True)
        target_var = torch.autograd.Variable(target,volatile=True)
        with torch.no_grad():
            # compute output
            output,ang_error = model(source_frame_var)

            loss = criterion(output, target_var,ang_error)
            angular_error = compute_angular_error(output,target_var)
            pred_error = ang_error[:,0]*180/math.pi
            pred_error = torch.mean(pred_error,0)

            angular.update(angular_error, source_frame.size(0))
            prediction_error.update(pred_error, source_frame.size(0))

            losses.update(loss.item(), source_frame.size(0))

            batch_time.update(time.time() - end)
            end = time.time()


        print('Epoch: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Angular {angular.val:.4f} ({angular.avg:.4f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
                    i, len(val_loader), batch_time=batch_time,
                   loss=losses,angular=angular))

    foo.add_scalar("predicted error", prediction_error.avg, count)
    foo.add_scalar("angular-test", angular.avg, count)
    foo.add_scalar("loss-test", losses.avg, count)
    return angular.avg




def save_checkpoint(state, is_best, filename='checkpoint_'+network_name+'.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best_'+network_name+'.pth.tar')


def spherical2cartesial(x):
    
    output = torch.zeros(x.size(0),3)
    output[:,2] = -torch.cos(x[:,1])*torch.cos(x[:,0])
    output[:,0] = torch.cos(x[:,1])*torch.sin(x[:,0])
    output[:,1] = torch.sin(x[:,1])

    return output


def compute_angular_error(input,target):

    input = spherical2cartesial(input)
    target = spherical2cartesial(target)

    input = input.view(-1,3,1)
    target = target.view(-1,1,3)
    output_dot = torch.bmm(target,input)
    output_dot = output_dot.view(-1)
    output_dot = torch.acos(output_dot)
    output_dot = output_dot.data
    output_dot = 180*torch.mean(output_dot)/math.pi
    return output_dot




class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count








if __name__ == '__main__':
    main()

validation.txt


In [5]:
!pip install tensorboardX

In [10]:
!pip install torchvision 

  Using cached torchvision-0.11.1-cp38-cp38-win_amd64.whl (984 kB)
  Using cached torch-1.10.0-cp38-cp38-win_amd64.whl (226.6 MB)


In [8]:
!conda activate env_pytorch

In [24]:
path = r"C:\Users\108652706\Documents\DOCUMENT_perso\ISI\MLA\trials\gaze360_model.pth.tar"

In [5]:
model_pred = torch.load(path,map_location=torch.device('cpu') )


In [25]:
model = GazeLSTM()
model = nn.DataParallel(model)
model.load_state_dict(torch.load(path,map_location=torch.device('cpu'))['state_dict'])

RuntimeError: Error(s) in loading state_dict for DataParallel:
	Missing key(s) in state_dict: "module.base_model.fc.weight", "module.base_model.fc.bias". 
	Unexpected key(s) in state_dict: "module.base_model.fc1.weight", "module.base_model.fc1.bias". 

problème de keys, donc j'ai trouvé ça sur internet pour créer un template du modèle apres copier coller les layers en fixant le problème des missing keys and unexpected keys.

In [29]:
model = GazeLSTM()
model = nn.DataParallel(model)
state_dict = torch.load('gaze360_model.pth.tar',map_location=torch.device('cpu'))["state_dict"]
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k# remove module.
    if (name =="module.base_model.fc1.weight"): 
        new_state_dict["module.base_model.fc.weight"] = v
    elif (name =="module.base_model.fc1.bias"): 
        new_state_dict["module.base_model.fc.bias"] = v
    else:
        new_state_dict[name] = v
    #print(k)
#load params

model.load_state_dict(new_state_dict)

<All keys matched successfully>

In [30]:
np.load()

DataParallel(
  (module): GazeLSTM(
    (base_model): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): B

In [21]:
#from torchsummary import summary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model2 = model.to(device)
#summary(model2,(224,244))
#print(torch_summarize(model))

In [8]:
image_normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

criterion = PinBallLoss().cuda(device)
test_loader = torch.utils.data.DataLoader(
            ImagerLoader(source_path,test_file,transforms.Compose([
                transforms.Resize((224,224)),transforms.ToTensor(),image_normalize,
            ])),
            batch_size=batch_size, shuffle=True,
            num_workers=workers, pin_memory=True)
#checkpoint = torch.load(checkpoint_test,map_location=torch.device('cpu'))
#       model.load_state_dict(checkpoint['state_dict'])
#angular_error = validate(test_loader, model, criterion)
#print('Angular Error is',angular_error)

test.txt


In [10]:
make_dataset('/imgs/','test.txt')

test.txt


[(['/imgs//rec_022/head/000000/000128.jpg',
   '/imgs//rec_022/head/000000/000129.jpg',
   '/imgs//rec_022/head/000000/000130.jpg',
   '/imgs//rec_022/head/000000/000131.jpg',
   '/imgs//rec_022/head/000000/000132.jpg',
   '/imgs//rec_022/head/000000/000133.jpg',
   '/imgs//rec_022/head/000000/000134.jpg'],
  array([ 0.45384066,  0.05778895, -0.889207  ])),
 (['/imgs//rec_022/head/000022/000128.jpg',
   '/imgs//rec_022/head/000022/000129.jpg',
   '/imgs//rec_022/head/000022/000130.jpg',
   '/imgs//rec_022/head/000022/000131.jpg',
   '/imgs//rec_022/head/000022/000132.jpg',
   '/imgs//rec_022/head/000022/000133.jpg',
   '/imgs//rec_022/head/000022/000134.jpg'],
  array([ 0.18733167,  0.03647325, -0.98161935])),
 (['/imgs//rec_022/head/000001/000128.jpg',
   '/imgs//rec_022/head/000001/000129.jpg',
   '/imgs//rec_022/head/000001/000130.jpg',
   '/imgs//rec_022/head/000001/000131.jpg',
   '/imgs//rec_022/head/000001/000132.jpg',
   '/imgs//rec_022/head/000001/000133.jpg',
   '/imgs//rec_0

In [12]:
img_path = "/imgs//rec_022/head/000584/000133.jpg"

In [19]:
im=cv2.imread(img_path)
print(im)

None


In [31]:
batch_time = AverageMeter()
losses = AverageMeter()
prediction_error = AverageMeter()
model.eval()
angular = AverageMeter()

In [ ]:
for i, (source_frame,target) in enumerate(test_loader):
        print(i)

In [45]:
img, label = test_loader[sample_idx]

TypeError: 'DataLoader' object is not subscriptable

In [44]:
test_loader

In [ ]:
list_o=[]
j=0
for i, (source_frame,target) in enumerate(test_loader):
        print(i, source_frame,target)
        try: 
            source_frame = source_frame.cuda(device,non_blocking=True)
            print("debug1")
            target = target.cuda(non_blocking=True)
            print("debug2")
            source_frame_var = torch.autograd.Variable(source_frame,volatile=True)
            target_var = torch.autograd.Variable(target,volatile=True)
            with torch.no_grad():
                # compute output
                output,ang_error = model(source_frame_var)
            list_o.append(output)
            print('one element done',j)
            j+=1
        except: 
            print('error accured')

In [ ]:
#import TensorDataset, DataLoader
 
inputs, classes = next(iter(test_loader)) 

In [12]:
source_frame = source_frame.cuda(device,non_blocking=True)

NameError: name 'source_frame' is not defined

In [ ]:
val_loader = torch.utils.data.DataLoader(
        ImagerLoader(source_path,val_file,transforms.Compose([
            transforms.Resize((224,224)),transforms.ToTensor(),image_normalize,
        ])),
        batch_size=batch_size, shuffle=True,
        num_workers=workers, pin_memory=True)



In [11]:
source_path

'/imgs/'